In [1]:
"""
author: Nora Helgeland
date: May, 2023

"""

import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy
import matplotlib as mpl
#import geopandas
import math
import cartopy as cr
#from mpl_toolkits.basemap import Basemap
from scipy.interpolate import griddata
import matplotlib.cbook as cbook
import cartopy.feature as cfeature
from numpy import nan
#import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams.update({'font.size': 12})
import re

In [13]:
df = pd.read_csv("/home/norah/master/filtered_stations.csv")
df


,1,2,3,4,5,6,7,8,9
0,ENAS,1004,78.92,11.93,8,20010101,99999999,NY-ALESUND II,NO
1,ENAN,1010,69.30,16.15,14,20140000,99999999,ANDOYA/ANDENES(AFB),NO
2,ENBJ,1028,74.52,19.02,18,19980000,99999999,BJORNOYA ISLAND,NO
3,ENBO,1152,67.25,14.40,8,19980000,99999999,BODO VI (CIV/MIL),NO
4,ENOL,1241,63.70,9.60,7,19980000,99999999,ORLAND III(NOR-AFB),NO
...,...,...,...,...,...,...,...,...,...
78,UKKK,33345,50.40,30.45,167,19970500,99999999,KIEV/ZHULYANY,UA
79,UKLL,33393,49.82,23.95,325,19950300,99999999,L'VIV,UA
80,9999,34009,51.77,36.17,247,19961200,99999999,KURSK,RU
81,UUOO,34122,51.67,39.27,104,19970500,99999999,VORONEZ,RU


In [2]:
def filter_stations(dataframe, lat_max, lat_min, long_max, long_min):

    
    newdf = dataframe[(dataframe[3].str.contains("N")) & (dataframe[4].str.contains("E"))] 
    
    #for latitude
    lat_list = []
    
    newdf = newdf.set_index(newdf[3])
    
    for i in newdf[3]:
        
        lat = i.replace('N', '')
        lat = float(lat)
        if lat_min > lat:
            lat_list.append(i)
        if lat_max < lat:
            lat_list.append(i)
    
    long_list = []
    newdf = newdf.drop(labels=lat_list, axis=0)
    newdf = newdf.set_index(newdf[4])  
    for i in newdf[4]:
        
        long = i.replace('E', '')
        long = float(long)
        if long_min > long:
            long_list.append(i)
        if long_max < long:
            long_list.append(i)
    
    newdf = newdf.drop(labels=long_list, axis = 0)
    newdf = newdf.set_index(pd.Index(range(0, np.size(newdf[4]), 1)))  
    lat_list2 = []
    long_list2 = []
    
    for i in newdf[3]:
        lat = i.replace('N', '')
        lat = float(lat)
        lat_list2.append(lat)
        
    for i in newdf[4]:
        long = i.replace('E', '')
        long = float(long)
        long_list2.append(long)
        
    lists = []
    lists.append(lat_list2)
    lists.append(long_list2)
    
    return lists
        
        

In [14]:
data = filter_stations(df, 58, 52, 40, 28)
data

KeyError: 3

In [90]:
# Add dummy column for plotting
df['Cs-137 kBq/m2']=10
data[0]

[78.92,
 69.3,
 74.52,
 67.25,
 63.7,
 60.2,
 58.87,
 59.95,
 65.55,
 63.18,
 62.53,
 57.67,
 57.65,
 67.37,
 62.4,
 60.82,
 57.1,
 56.3,
 55.77,
 52.1,
 54.53,
 53.72,
 54.1,
 53.38,
 52.82,
 53.2,
 52.73,
 52.47,
 52.12,
 52.22,
 51.4,
 51.13,
 51.55,
 51.12,
 51.17,
 54.75,
 52.4,
 51.13,
 68.98,
 68.97,
 67.15,
 64.98,
 61.82,
 59.45,
 59.97,
 59.95,
 57.8,
 57.9,
 56.97,
 56.65,
 56.88,
 56.38,
 54.88,
 54.7,
 54.75,
 53.87,
 52.12,
 52.45]

In [2]:
# %matplotlib widget
#fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()}, figsize=(10,15))
df = pd.read_csv("/home/norah/master/filtered_stations.csv")
fig,ax = plt.subplots(subplot_kw={'projection':ccrs.LambertConformal(central_longitude=10, standard_parallels=(54,78))}, figsize=(10,15))
ax.scatter(30.1, 51.4, color='blue', transform=ccrs.PlateCarree())
ax.scatter(df["4"], df["3"], color='red', transform=ccrs.PlateCarree())
gl = ax.gridlines(draw_labels=True)
gl.top_labels=False
gl.right_labels=False

#borders,lakes,ocean,coastline,rivers and so on 
provinces_50m = cfeature.NaturalEarthFeature('cultural',
                                             'admin_1_states_provinces_lines',
                                             '50m',
                                             facecolor='none')

#ax.set_extent((-20, 80, 51, 88), crs=ccrs.PlateCarree())
ax.set_extent((-20, 60, 38, 75))
ax.coastlines()
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.LAKES)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.RIVERS)
ax.legend(['Chernobyl', 'Radiosonde Stations'])

#ax.add_feature(provinces_50m)


ax.set_title('All Radiosonde Locations', fontsize=14)
plt.savefig('sonder.png')